# MSDS 7331 Data Mining Lab 1
Profesor: Dr. Jake Drew  
Team: Steven Hayden, Josephine MacDaniel, Afreen Siddiqui, & Eduardo Cantu

## Business Understanding


## Data Meaning Type

## Data Quality

## Simple Statistics

## Visualize Attributes

## Explore Joint Attributes

## Explore Attributes and Class

## New Features

## Exceptional Work